<a href="https://colab.research.google.com/github/PRomanVl/deep_learning_school/blob/main/virtual_try_on/virtual_try_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Virtual try on

#    Загрузка FrankMocap

In [ ]:
!rm -rf {,.[!.],..?}*

In [ ]:
!git clone --quiet https://github.com/facebookresearch/frankmocap.git .
!apt-get install -qq xvfb libosmesa6-dev
!pip install -q -r docs/requirements.txt
!pip install -q 'git+https://github.com/facebookresearch/pytorch3d.git@stable'
!sh scripts/install_pose2d.sh
!sh scripts/download_data_body_module.sh
!sh scripts/download_sample_data.sh
!pip install -q torchgeometry

Selecting previously unselected package xvfb.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Selecting previously unselected package libosmesa6:amd64.
Preparing to unpack .../libosmesa6_20.0.8-0ubuntu1~18.04.1_amd64.deb ...
Unpacking libosmesa6:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Selecting previously unselected package libosmesa6-dev:amd64.
Preparing to unpack .../libosmesa6-dev_20.0.8-0ubuntu1~18.04.1_amd64.deb ...
Unpacking libosmesa6-dev:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Setting up xvfb (2:1.19.6-1ubuntu4.9) ...
Setting up libosmesa6:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Setting up libosmesa6-dev:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

    

In [ ]:
!mkdir -p extra_data/smpl
# !wget -nc http://smplify.is.tue.mpg.de/main/download1 -O mpips-smplify_public_v2.zip
!test -f smplify_public_v2.zip || gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws # https://drive.google.com/file/d/1BwNtKvArmUdy4Cpkx1seIlsHinopdaws/view?usp=sharing
!unzip -n -p mpips-smplify_public_v2.zip smplify_public/code/models/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl > extra_data/smpl/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl

Downloading...
From: https://drive.google.com/uc?id=1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
To: /content/mpips-smplify_public_v2.zip
53.0MB [00:00, 145MB/s]


Загрузка MultiGarmentNetwork

In [ ]:
!git clone --quiet https://github.com/bharat-b7/MultiGarmentNetwork.git /content/MGN

Download digital wardrobe: [Part-1](https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset.zip) and [Part-2](https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset_02.zip).
This dataset contains scans, SMPL registration, texture_maps, segmentation_maps and multi-mesh registered garments.

In [ ]:
!wget -P ./MGN https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset.zip 

In [ ]:
!unzip /content/MGN/Multi-Garmentdataset.zip -d /content/MGN

In [ ]:
!wget -P ./MGN https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset_02.zip 

--2021-06-28 19:17:44--  https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset_02.zip
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72520880 (69M) [application/zip]
Saving to: ‘./MGN/Multi-Garmentdataset_02.zip’

Multi-Garmentdatase 100%[===================>]  69.16M  18.9MB/s    in 4.6s    

2021-06-28 19:17:50 (14.9 MB/s) - ‘./MGN/Multi-Garmentdataset_02.zip’ saved [72520880/72520880]



In [ ]:
!unzip /content/MGN/Multi-Garmentdataset_02.zip -d /content/MGN

Download the neutral SMPL model from http://smplify.is.tue.mpg.de/ and place it in the assets folder.

In [ ]:
!mkdir -p ./MGN/assets
!test -f mpips-smplify_public_v2.zip || gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws > ./MGN
!test -f assets/neutral_smpl.pkl || unzip -n -p mpips-smplify_public_v2.zip smplify_public/code/models/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl > ./MGN/assets/neutral_smpl.pkl

In [ ]:
!gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws 

Downloading...
From: https://drive.google.com/uc?id=1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
To: /content/mpips-smplify_public_v2.zip
53.0MB [00:00, 144MB/s]


Download and install DIRT: https://github.com/pmh47/dirt.

In [ ]:
!git clone --quiet https://github.com/pmh47/dirt.git ./MGN/dirt_dir
!sed -i 's|set(CMAKE_CUDA_FLAGS "${CMAKE_CUDA_FLAGS} -arch=sm_30 --expt-relaxed-constexpr -DNDEBUG")|set(CMAKE_CUDA_FLAGS "${CMAKE_CUDA_FLAGS} -arch=sm_60 --expt-relaxed-constexpr -DNDEBUG")|' ./MGN/dirt_dir/csrc/CMakeLists.txt
!cd dirt_dir && pip install -q ./MGN

/bin/bash: line 0: cd: dirt_dir: No such file or directory


Download and install mesh packages for visualization: https://github.com/MPI-IS/mesh

In [ ]:
!git clone --quiet https://github.com/MPI-IS/mesh.git ./MGN/mesh_dir

In [ ]:
!apt-get install -qq libboost-dev xvfb libosmesa6-dev

In [ ]:
!cd MGN/mesh_dir && make all

----- [ mesh_package ] Installing with the interpreter /usr/local/bin/python (version 3.7.10)
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 1)) (57.0.0)
     |████████████████████████████████| 15.7MB 350kB/s 
     |████████████████████████████████| 10.3MB 29.4MB/s 
     |████████████████████████████████| 28.5MB 128kB/s 
Requirement already up-to-date: pyopengl in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 5)) (3.1.5)
     |████████████████████████████████| 3.0MB 31.6MB/s 
Requirement already up-to-date: pyzmq in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 7)) (22.1.0)
Requirement already up-to-date: pyyaml in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 8)) (5.4.1)
     |████████████████████████████████| 51.0MB 98kB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datas

In [ ]:
!pip install -q chumpy opendr

Patch Python 2 to Python 3 and fix hardcoded paths

In [ ]:
!find ./ -type f -name "*.py" -exec sed -i 's|import cPickle as pkl|import _pickle as pkl|' {} \;
!find ./ -type f -name "*.py" -exec sed -i 's|import cPickle as pickle|import _pickle as pickle|' {} \;
!find ./ -type f -name "*.py" -exec sed -i -r s/pkl\.load[\(]open[\(]\(.+\)[\)][\)]/pkl.load\(open\(\\1,\ \'rb\'\)\ ,\ encoding=\'latin1\'\)/g  {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|/BS/bharat/work/MGN_release/||" {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|/BS/bharat/work/MGN_final_release/||" {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|from posemapper|from .posemapper|" {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|from serialization|from .serialization|" {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|from verts|from .verts|" {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|^import lbs$|from . import lbs|" {} \;
!find ./ -type f -name "*.py" -exec sed -i "s|/BS/RVH/work/data/smpl_models/neutral/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl|assets/neutral_smpl.pkl|" {} \;

In [ ]:
!find ./ -type f -name "*.py" -exec sed -i "s|assets/|MGN/assets/|" {} \;

In [ ]:
from IPython.display import Image

In [ ]:
# !sed -i "s|scan2.show()|scan2.show().save_snapshot('scan.png', blocking=True)|" /content/MGN/visualize_scan.py

In [ ]:
# !sed -i "s|mvs\[0\]\[1\].set_static_meshes(\[garment_org_body\])|mvs\[0\]\[1\].set_static_meshes(\[garment_org_body\]); mvs\[0\]\[1\].save_snapshot('garment_org_body.png', blocking=True)|" /content/MGN/visualize_garments.py
# !sed -i "s|mvs\[0\]\[2\].set_static_meshes(\[garment_org_body, garment_posed\])|mvs\[0\]\[2\].set_static_meshes(\[garment_org_body, garment_posed\]); mvs\[0\]\[2\].save_snapshot('garment_org_body_garment_posed.png', blocking=True)|" /content/MGN/visualize_garments.py
# !sed -i "s|mvs\[0\]\[0\].set_static_meshes(\[garment_unposed\])|mvs\[0\]\[0\].set_static_meshes(\[garment_unposed\]); mvs\[0\]\[0\].save_snapshot('garment_unposed.png', blocking=True)|" /content/MGN/visualize_garments.py

Load random garment and dress desired SMPL body with it.

In [ ]:
# !sed -i "s|mvs\[0\]\[0\].set_static_meshes(\[garment_unposed\])|mvs\[0\]\[0\].set_static_meshes(\[garment_unposed\]); mvs\[0\]\[0\].save_snapshot('dress_unposed.png', blocking=True)|" MGN/dress_SMPL.py
# !sed -i "s|mvs\[0\]\[1\].set_static_meshes(\[new_garment, tgt_body\])|mvs\[0\]\[1\].set_static_meshes(\[new_garment, tgt_body\]); mvs\[0\]\[1\].save_snapshot('dress_posed.png', blocking=True)|" MGN/dress_SMPL.py

Загрузка измененного кода

In [ ]:
#dress_SMPL
!gdown --id 1U-0je1wjPPKa6dZRBj98pgLijjK1RJHi 
!rm ./MGN/dress_SMPL.py
!mv '/content/dress_SMPL.py' '/content/MGN/dress_SMPL.py'
#smpl_path
!gdown --id 1bEee9xihGg3YYGIhFlLSoErt-01LM2fC
!rm ./MGN/utils/smpl_paths.py
!mv '/content/smpl_paths.py' './MGN/utils/smpl_paths.py'
#screen_free_visualizer
!gdown --id 18AIisb-ye96UHJW2d3kXG9lq_diOgN1K
!rm ./renderer/screen_free_visualizer.py
!mv '/content/screen_free_visualizer.py' './renderer/screen_free_visualizer.py'
#p3d_render
!gdown --id 1K9Pz-uIgE9pOykh-xupm0WnT26mLoFZi
!rm ./renderer/p3d_renderer.py
!mv '/content/p3d_renderer.py' './renderer/p3d_renderer.py'
#interpenetration_ind.py
!gdown --id 18z-baZAHJOkUDN9gVU2Q-D2Z9r1M-DlC
!rm ./MGN/utils/interpenetration_ind.py
!mv '/content/interpenetration_ind.py' './MGN/utils/interpenetration_ind.py'
#demo_options.py
!gdown --id 1cG8r8WW41PjCMn8HLdp3O5Asuk5NHMby
!rm ./demo/demo_options.py
!mv '/content/demo_options.py' './demo/demo_options.py'
#demo_bodymocap.py
!gdown --id 1tPB89slpuz2AKs-VoGvJBl2U0JaREfih
!rm ./demo/demo_bodymocap.py
!mv '/content/demo_bodymocap.py' './demo/demo_bodymocap.py'
#ch_smpl.py
!gdown --id 1kiAxIHXU6eOBFzU0rrXuEnsewIh-mktp
!rm ./MGN/lib/ch_smpl.py
!mv '/content/ch_smpl.py' '/content/MGN/lib/ch_smpl.py'

Downloading...
From: https://drive.google.com/uc?id=1U-0je1wjPPKa6dZRBj98pgLijjK1RJHi
To: /content/dress_SMPL.py
100% 6.35k/6.35k [00:00<00:00, 5.56MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bEee9xihGg3YYGIhFlLSoErt-01LM2fC
To: /content/smpl_paths.py
100% 2.74k/2.74k [00:00<00:00, 2.34MB/s]
Downloading...
From: https://drive.google.com/uc?id=18AIisb-ye96UHJW2d3kXG9lq_diOgN1K
To: /content/screen_free_visualizer.py
100% 3.61k/3.61k [00:00<00:00, 6.31MB/s]
Downloading...
From: https://drive.google.com/uc?id=1K9Pz-uIgE9pOykh-xupm0WnT26mLoFZi
To: /content/p3d_renderer.py
100% 6.92k/6.92k [00:00<00:00, 11.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=18z-baZAHJOkUDN9gVU2Q-D2Z9r1M-DlC
To: /content/interpenetration_ind.py
100% 4.51k/4.51k [00:00<00:00, 7.50MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cG8r8WW41PjCMn8HLdp3O5Asuk5NHMby
To: /content/demo_options.py
100% 4.65k/4.65k [00:00<00:00, 7.71MB/s]
Downloading...
From: https://drive.google.com/uc?i

# Запуск virtual try on

In [ ]:
!xvfb-run -a python -m demo.demo_bodymocap --input_path ./sample_data/han_short.mp4 --out_dir ./mocap_output

Loading Body Pose Estimator
use_smplx False
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:00<00:00, 135MB/s]
--------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Visualization saved: ./mocap_output/rendered/00000.jpg
Time: 16.55 sec/frame, FPS 0.06
Processed : ./mocap_output/frames/00000.jpg
--------------------------------------
Visualization saved: ./mocap_output/rendered/00001.jpg
Time: 14.95 sec/frame, FPS 0.07
Processed : ./mocap_output/frames/00001.jpg
--------------------------------------


In [ ]:
# source: https://stackoverflow.com/a/65273831

from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "mocap_output/han_short.mp4"

# Compressed video path
compressed_path = "mocap_output/han_short_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)